In [62]:
import numpy as np
from numpy.linalg import inv
from tkinter import *

In [63]:
class KalmanFilter:
    """
    Simple Kalman filter
    """

    def __init__(self, X, F, Q, Z, H, R, P, B=np.array([0]), M=np.array([0])):
        """
        Initialise the filter
        Args:
            X: State estimate
            P: Estimate covariance
            F: State transition model
            B: Control matrix
            M: Control vector
            Q: Process noise covariance
            Z: Measurement of the state X
            H: Observation model
            R: Observation noise covariance
        """
        self.X = X
        self.P = P
        self.F = F
        self.B = B
        self.M = M
        self.Q = Q
        self.Z = Z
        self.H = H
        self.R = R

    def predict(self):
        """
        Predict the future state
        Args:
            self.X: State estimate
            self.P: Estimate covariance
            self.B: Control matrix
            self.M: Control vector
        Returns:
            updated self.X
        """
        # Project the state ahead
        self.X = self.F @ self.X + self.B @ self.M
        self.P = self.F @ self.P @ self.F.T + self.Q

        return self.X

    def update(self, Z):
        """
        Update the Kalman Filter from a measurement
        Args:
            self.X: State estimate
            self.P: Estimate covariance
            Z: State measurement
        Returns:
            updated X
        """
        K = self.P @ self.H.T @ inv(self.H @ self.P @ self.H.T + self.R)
        self.X += K @ (Z - self.H @ self.X)
        self.P = self.P - K @ self.H @ self.P

        return self.X


In [64]:
stateMatrix = np.zeros((4, 1))  # [x, y, delta_x, delta_y]
estimateCovariance = np.eye(stateMatrix.shape[0])
transitionMatrix = np.array(
    [[1, 0, 1, 0],
     [0, 1, 0, 1], 
     [0, 0, 1, 0], 
     [0, 0, 0, 1]])
processNoiseCov = np.array([
    [1, 0, 0, 0], 
    [0, 1, 0, 0], 
    [0, 0, 1, 0], 
    [0, 0, 0, 1]]) * 0.001
measurementStateMatrix = np.zeros((2, 1))
observationMatrix = np.array([
    [1, 0, 0, 0], 
    [0, 1, 0, 0]])
measurementNoiseCov = np.array([
    [1, 0], 
    [0, 1]]) * 1
kalman = KalmanFilter(X=stateMatrix,
                      P=estimateCovariance,
                      F=transitionMatrix,
                      Q=processNoiseCov,
                      Z=measurementStateMatrix,
                      H=observationMatrix,
                      R=measurementNoiseCov)


In [65]:
class Points:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"

    

In [69]:
app = Tk()

# lastx, lasty = None, None 
# xnoisy, ynoisy = None, None

canvas = Canvas(app, width=640, height=480, bg='black')
canvas.pack(anchor='nw', fill='both', expand=1)
realcords = []
noiseCords = []
filterCords = []


def get_x_and_y(event):
    global lasx, lasy
    global laspx, laspy

    lasx, lasy = event.x, event.y
    laspx, laspy = event.x+np.random.randint(-30,30), event.y+np.random.randint(-30,30)




def draw(event):
    global lasx, lasy
    global laspx, laspy
    global predict, measure
       
    canvas.create_line((lasx, lasy, event.x, event.y),fill='red',width=2)
    realcords.append(Points(event.x,event.y))
    # noise points
    xnoise = event.x+np.random.randint(-30,30)
    ynoise = event.y+np.random.randint(-30,30)
    canvas.create_line(xnoise, ynoise, xnoise+1, ynoise+1, fill='green')
    noiseCords.append(Points(xnoise,ynoise))
    measure = np.array([[xnoise],[ynoise]])

    #plot prediction 
    predict = kalman.predict()
    xpred = predict[0, 0].round()
    ypred = predict[1, 0].round()
    canvas.create_line(laspx, laspy, xpred, ypred, fill='white')
    # print(predict[0,0],predict[1,0], lasx, lasy)
    kalman.update(measure)

    laspx, laspy = xpred, ypred
    lasx, lasy = event.x, event.y
    


def release(event):
    print(realcords)
    # print(xcords,ycords)

canvas.bind("<Button-1>", get_x_and_y)
canvas.bind("<B1-Motion>", draw)
canvas.bind("<ButtonRelease-1>", release)

app.mainloop()


[(174,195), (175,195), (175,194), (176,194), (177,194), (177,193), (178,193), (179,193), (179,192), (180,192), (181,192), (182,191), (183,191), (184,191), (185,190), (186,190), (187,190), (188,190), (189,190), (189,189), (191,189), (192,189), (193,189), (194,189), (195,189), (195,188), (196,188), (197,188), (198,188), (199,188), (199,187), (200,187), (201,187), (202,187), (203,187), (204,187), (205,187), (206,187), (207,187), (208,187), (209,187), (210,187), (211,187), (212,187), (213,187), (214,187), (215,187), (216,187), (217,187), (218,187), (219,187), (220,187), (221,187), (222,187), (223,187), (224,187), (225,187), (227,187), (228,187), (229,187), (230,187), (231,187), (232,187), (233,187), (234,187), (235,187), (236,187), (237,187), (238,187), (239,187), (240,187), (241,187), (242,187), (242,186), (245,186), (246,186), (247,186), (247,185), (248,185), (249,185), (250,185), (251,185), (252,185), (253,185), (254,185), (255,185), (256,185), (257,185), (257,184), (258,184), (259,184)